In [16]:
pip install -U scikit-learn

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.6/dist-packages (0.24.2)
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install dictances

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [18]:
from __future__ import division
import json
import numpy as np
from decimal import Decimal
from numpy import expand_dims, array, exp, max
import scipy
from scipy.special import rel_entr
import csv
import statistics
from numpy import expand_dims, array, exp, max
import math
from scipy.spatial import distance
from scipy.stats import wasserstein_distance
import numpy as np
from numpy.linalg import norm
import sklearn
from tensorflow import keras
import string
from scipy.special import rel_entr
from tensorflow.keras.applications.resnet50 import ResNet50 as resnet50
from tensorflow.keras.applications.vgg16 import VGG16 as vgg16
from tensorflow.keras.applications.densenet import DenseNet169 as densenet
from tensorflow.keras.applications.inception_v3 import InceptionV3 as inceptionv3
from tensorflow.keras.applications.efficientnet import EfficientNetB1 as efficient
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 as mobilenetv2
from tensorflow.keras.applications.mobilenet import MobileNet as mobilenet
from sklearn.metrics.pairwise import cosine_similarity
from itertools import combinations
import csv
# import the timeit module
import timeit
import tracemalloc

In [19]:
from numpy.linalg import norm
from dictances import bhattacharyya, bhattacharyya_coefficient

In [20]:
# import the timeit module
import timeit
import math
import tracemalloc

In [21]:
def hellinger_fast(p, q):
    """Hellinger distance between two discrete distributions.
       In pure Python.
       Fastest version.
    """
    return sum([ (math.sqrt(p_i) - math.sqrt(q_i))**2 for p_i, q_i in zip(p,q) ])

In [22]:
# to suppress division errors
np.seterr(divide='ignore', invalid='ignore')
new_final_source_array = []

In [23]:
# Opening final JSON files
contrast_source_data = open('../paper3/files/glcm/source_domain_data_new_energy_glcm.json')

#for target data
contrast_target_data = open('files/Caltech256/InceptionV3/glcm/caltech_domain_data_new_energy_glcm.json')

In [24]:
# returns JSON object
contrast_source_data_d = json.load(contrast_source_data)

contrast_target_data_d = json.load(contrast_target_data)

In [25]:
# kdl dictionary
kdl_dictionary = {}

In [26]:
def KL(p, q):
    a = []
    b = []
    for k in p:
        a.append(Decimal(k))

    for l in q:
        b.append(Decimal(l))

    a = np.asarray(a, dtype=np.float64)
    b = np.asarray(b, dtype=np.float64)

    return np.sum(np.where(((a != 0) & (b != 0)), a * np.log(a / b), 0))

In [27]:
def softMax(temperature, features_list):
    feat_list = [x / temperature for x in features_list]
    feature_list = exp(feat_list - max(feat_list))
    return feature_list / feature_list.sum(axis=0)

In [28]:
# get the data for the contrast target image
contrast_target_data_array = []
for p in range(0, len(contrast_target_data_d['target_images'])):
    for i, j in contrast_target_data_d['target_images'][p].items():
        contrast_target_data_array.append((i,np.array(j)))
contrast_target_data.close()

In [29]:
print(len(contrast_target_data_array))

7710


In [31]:
###contrast comparison code
contrast_All_Images = []
contrast_average_dkl = []

#check the timer_start
start_dkl = timeit.default_timer()
#check memory consumption
# starting the monitoring
tracemalloc.start()

for x in contrast_target_data_array[:1]:
#for x in target_data_array:
    t_images_btween_0_05 = []
    t_images_btween_05_1 = []
    t_images_btween_1_2 = []
    t_images_grt_2_5 = []
    t_images_grt_5_10 = []
    t_images_grt_10 = []
    
    image_name_class =  x[0].split('_/') #name[0]; class[1]
    compare_min_max = []
    
    
            
    # get the data for the source images
    for m in range(0, len(contrast_source_data_d['source_images'])):
        #print("source",source_data_d['source_images'][m][0])
        for k, v in contrast_source_data_d['source_images'][m].items():
            # compare the value of v with that of the target image
            # if 'caterpillar' in k.lower():
            # remove the zeros from the list
            new_v = np.array(v).flatten()
            #get the top items in x
            items = np.sort(x[1])
            item_size = items[-len(new_v):]
            new_x = item_size.flatten()
            
            new_d = softMax(0.5, new_x)     
            new_y = softMax(0.5, new_v)
            
            kdl = rel_entr( new_x, new_v, out=None)
            new_bhattacharyya = bhattacharyya(dict(enumerate(new_x.flatten(), 1)),dict(enumerate(new_v.flatten(), 1)) )
            new_wasserstein = wasserstein_distance(new_x,new_v)
            new_jensen = distance.jensenshannon(new_x,new_v)
            new_hellinger = hellinger_fast(new_x,new_v)
            
            compare_min_max.append((image_name_class[0],image_name_class[1],k, max(kdl)))
            
contrast_All_Images.append(compare_min_max)
    
contrast_source_data.close()

stop = timeit.default_timer()
execution_time = stop - start_dkl
print("KDL -selected points execution time:-", execution_time)

# displaying the memory
print("used memory",tracemalloc.get_traced_memory()[1]-tracemalloc.get_traced_memory()[0])
 
# stopping the library
tracemalloc.stop()

#check the layers
model_name = resnet50
input_t = keras.Input(shape=(224, 224, 3))
model = model_name(include_top=False,
                   weights="imagenet",
                   input_tensor=input_t)


# get the layer index
def getLayerIndex(model_i, layer_name):
    for pos, layer_g in enumerate(model_i.layers):
        if layer_g.name == layer_name:
            return pos


# get the convolved layers into an array for looping
convolved_layers = []
convolved_layer_names = []
# create a list to place them
layer_cosine_list = []
layer_final_cos_list = []

for layer in model.layers[0:-2]:
    if 'conv' not in layer.name:
        continue

    index = getLayerIndex(model, layer.name)
    #print(layer.name)
    # append the convolved layer
    convolved_layers.append(index)
    convolved_layer_names.append(layer)

jdk = {}
for lyr in convolved_layer_names:
    ary = np.array(lyr.get_weights(), dtype=object)
    layer_index = getLayerIndex(model, lyr.name)
    if len(ary) != 0:
        # filters, biases = layer.get_weights()
        # print(filters.shape)
        ary = np.array(lyr.get_weights(), dtype=object)

        zipper_dict = {}
        dict_array = []

        # check for the arrays
        for x in ary:
            # print(len(x))
            # find if the array is 1 dim
            if x.ndim > 1:
                for y in x:
                    for z in y:
                        u = 0
                        for za in z:
                            # get all the positive elements in this and push them into an array
                            positive_elements = []
                            for pos_item in za:
                                if pos_item > 0:
                                    positive_elements.append(pos_item)
                            u = u + 1
                            zipper_dict.update({u: positive_elements})

            elif x.ndim == 1:
                # get the items/arrays with more than the 1 dimension
                print("")
        # print("lyr:",lyr.name)
        # get two items per tuple
        for i in range(1, len(zipper_dict), 2):
            if len(zipper_dict[i]) > 0:
                if len(zipper_dict[i]) == len(zipper_dict[i + 1]):

                    new_i = np.array(zipper_dict[i]).reshape(1, -1)
                    new_i_plus_1 = np.array(zipper_dict[i + 1]).reshape(1, -1)
                    cos_item = np.ravel(cosine_similarity(new_i, new_i_plus_1))
                    layer_cosine_list.append(cos_item)

                elif len(zipper_dict[i]) > len(zipper_dict[i + 1]):

                    diff_len = len(np.array(zipper_dict[i])) - len(np.array(zipper_dict[i + 1]))
                    old_i = np.pad(zipper_dict[i + 1], (0, diff_len), 'constant')

                    new_i = old_i.reshape(1, -1)
                    new_i_plus_1 = np.array(zipper_dict[i]).reshape(1, -1)

                    cos_item = np.ravel(cosine_similarity(new_i, new_i_plus_1))
                    layer_cosine_list.append(cos_item)

                elif len(zipper_dict[i]) < len(zipper_dict[i + 1]):
                    # print("######less than i plus 1#########")

                    diff_len = len(zipper_dict[i + 1]) - len(zipper_dict[i])
                    old_i = np.pad(zipper_dict[i], (0, diff_len), 'constant')
                    new_i = old_i.reshape(1, -1)
                    new_i_plus_1 = np.array(zipper_dict[i + 1]).reshape(1, -1)

                    cos_item = np.ravel(cosine_similarity(new_i, new_i_plus_1))
                    layer_cosine_list.append(cos_item)


        # calculate softmax function
        def softmax(s_et):
            e_set = np.exp(s_et - np.max(s_et))
            return e_set / e_set.sum(axis=0)


        for a in layer_cosine_list:
            layer_final_cos_list.append(a)
        # print(layer_final_cos_list)
        layer_softmax_values = softmax(np.ravel(np.array(layer_cosine_list, dtype=np.float32).reshape(1, -1)))
        jdk.update({layer_index: layer_softmax_values})

# jdk gives a dictionary made up of probability distributions
# print(jdk)
# print("the other one is")
b = [x for y, x in enumerate(jdk) if y != i]
all_items = list(combinations(b, 2))


# put the layers in a dictionary and rank them
other = []

# start the timer
start_kdl1 = timeit.default_timer()
#check memory consumption
# starting the monitoring
tracemalloc.start()

for a in all_items:
    if len(jdk[a[0]]) > len(jdk[a[1]]):
        dif_1 = len(np.array(jdk[a[0]])) - len(np.array(jdk[a[1]]))
        new_jdk = np.pad(np.array(jdk[a[1]]), (0, dif_1), 'constant').reshape(1, -1)
        new_i_jdk = np.array(jdk[a[0]]).reshape(1, -1)       
        #print("########################other###############") 
        kl_pq = rel_entr(np.ravel(np.ravel(new_i_jdk), new_jdk))
        #print(sum(kl_pq))
        #print("##########################end other#########")
       # print("layer:", a[0], " and layer: ", a[1], ':KL(P ||2 Q): %.3f nats1' % sum(kl_pq))
        #m_no = [a[0], a[1], sum(kl_pq)]
        #other.append(m_no)

    elif len(jdk[a[0]]) < len(jdk[a[1]]):
        dif_1 = len(np.array(jdk[a[1]])) - len(np.array(jdk[a[0]]))
        new_jdk = np.pad(np.array(jdk[a[0]]), (0, dif_1), 'constant').reshape(1, -1)
        new_i_jdk = np.array(jdk[a[1]]).reshape(1, -1)
        #print("########################other###############") 
        kl_pq = rel_entr(np.ravel(new_jdk), np.ravel(new_i_jdk))
        print(sum(kl_pq))
        #print("##########################end other#########")

#stop the execution
stop_dkl1 = timeit.default_timer()
execution_time = stop_dkl1 - start_dkl1
print(" execution time:-", execution_time)

# displaying the memory
print(" memory",tracemalloc.get_traced_memory()[1]-tracemalloc.get_traced_memory()[0])
 
# stopping the library
tracemalloc.stop()


KDL -selected points execution time:- 3.754144554026425
used memory 9666789









































































































































































































































































3.541674852371216
3.541674852371216
4.2181291580200195
4.2181291580200195
4.597789287567139
4.877163887023926
4.877163887023926
4.877163887023926
5.607540130615234
5.607540130615234
5.732232570648193
5.732232570648193
5.833760738372803
5.833760738372803
6.190915107727051
6.190915107727051
6.2641191482543945
6.2641191482543945
6.322389602661133
6.322389602661133
6.544638156890869
6.544638156890869
6.642646312713623
6.642646312713623
6.802158832550049
6.8718438148498535
6.8718438148498535
6.8718438148498535
7.128078937530518
7.128078937530518
7.1851701736450195
7.1851701736450195
7.232760429382324
7.232760429382324
7.414540767669678
7.414540767669678
7.

3.2366948388516903
3.2366948388516903
3.2704542838037014
3.2704542838037014
3.4091470167040825
3.4091470167040825
3.442484464496374
3.442484464496374
3.4723531380295753
3.4723531380295753
3.5919129624962807
3.5919129624962807
3.6480759978294373
3.6480759978294373
3.7488864846527576
3.7958258502185345
3.7958258502185345
3.7958258502185345
3.9661705382168293
3.9661705382168293
4.005219429731369
4.005219429731369
4.041527342051268
4.041527342051268
4.1756338365376
4.1756338365376
4.207759656012058
4.207759656012058
4.237642098218203
4.237642098218203
4.350979823619127
4.350979823619127
4.377923335880041
4.377923335880041
4.402400858700275
4.402400858700275
4.498383719474077
4.498383719474077
4.5214435793459415
4.5214435793459415
4.5429933071136475
4.5429933071136475
4.627378880977631
4.627378880977631
4.647648137062788
4.647648137062788
4.666860330849886
4.666860330849886
4.743252065032721
4.743252065032721
4.780289579182863
4.780289579182863
4.847847998142242
4.879804011434317
4.87980401

4.341385757550597
4.369919739663601
4.369919739663601
4.395990028977394
4.395990028977394
4.497700285166502
4.497700285166502
4.522039337083697
4.522039337083697
4.544100562110543
4.544100562110543
0.730376226361841
0.730376226361841
0.8550689239054918
0.8550689239054918
0.9565970567055047
0.9565970567055047
1.313751167152077
1.313751167152077
1.3869552197866142
1.3869552197866142
1.4452255265787244
1.4452255265787244
1.66747435182333
1.66747435182333
1.7654823325574398
1.7654823325574398
1.9249948421493173
1.9946797881275415
1.9946797881275415
1.9946797881275415
2.2509150626137853
2.2509150626137853
2.308006069622934
2.308006069622934
2.355596637353301
2.355596637353301
2.537376733496785
2.537376733496785
2.577660297974944
2.577660297974944
2.611419729888439
2.611419729888439
2.7501124693080783
2.7501124693080783
2.7834498956799507
2.7834498956799507
2.813318577595055
2.813318577595055
2.9328784216195345
2.9328784216195345
2.989041430875659
2.989041430875659
3.0898519549518824
3.13679

2.8368763416074216
2.863819867838174
2.863819867838174
2.8882973957806826
2.8882973957806826
2.9842802970670164
2.9842802970670164
3.007340104319155
3.007340104319155
3.028889868874103
3.028889868874103
3.113275449257344
3.113275449257344
3.133544697891921
3.133544697891921
3.152756893541664
3.152756893541664
3.2291486198082566
3.2291486198082566
3.2661860841326416
3.2661860841326416
3.333744578063488
3.3657005662098527
3.3657005662098527
3.3657005662098527
3.4863168792799115
3.4863168792799115
3.5148508260026574
3.5148508260026574
3.540921143256128
3.540921143256128
3.642631405033171
3.642631405033171
3.6669704345986247
3.6669704345986247
3.689031676389277
3.689031676389277
0.3571541109122336
0.3571541109122336
0.4303581670974381
0.4303581670974381
0.4886284734820947
0.4886284734820947
0.7108773033833131
0.7108773033833131
0.8088852816727012
0.8088852816727012
0.9683977984823287
1.03808274702169
1.03808274702169
1.03808274702169
1.2943180315196514
1.2943180315196514
1.3514090373646468

2.581389147322625
2.581389147322625
2.601658402942121
2.601658402942121
2.6208705911412835
2.6208705911412835
2.697262328118086
2.697262328118086
2.73429977404885
2.73429977404885
2.8018582784570754
2.8338142661377788
2.8338142661377788
2.8338142661377788
2.9544305792078376
2.9544305792078376
2.982964531518519
2.982964531518519
3.0090348394587636
3.0090348394587636
3.1107451049610972
3.1107451049610972
3.1350841247476637
3.1350841247476637
3.157145371194929
3.157145371194929
0.05827030586078763
0.05827030586078763
0.2805191330553498
0.2805191330553498
0.3785271105589345
0.3785271105589345
0.5380396238178946
0.607724575267639
0.607724575267639
0.607724575267639
0.863959850044921
0.863959850044921
0.9210508582182229
0.9210508582182229
0.9686414237366989
0.9686414237366989
1.1504215283785015
1.1504215283785015
1.1907050885492936
1.1907050885492936
1.224464524188079
1.224464524188079
1.3631572573212907
1.3631572573212907
1.3964946829946712
1.3964946829946712
1.4263633692171425
1.4263633692

...

1.1997721236548387
1.231728115642909
1.231728115642909
1.231728115642909
1.3523444381717127
1.3523444381717127
1.3808783840504475
1.3808783840504475
1.4069487045926508
1.4069487045926508
1.5086589733255096
1.5086589733255096
1.5329979976231698
1.5329979976231698
1.5550592423824128
1.5550592423824128
0.046939336393734266
0.046939336393734266
0.046939336393734266
0.21728403491943027
0.21728403491943027
0.2563329263430205
0.2563329263430205
0.29264083859015955
0.29264083859015955
0.42674732015439076
0.42674732015439076
0.458873147435952
0.458873147435952
0.48875554912228836
0.48875554912228836
0.6020932767278282
0.6020932767278282
0.6290368080663029
0.6290368080663029
0.6535143277869793
0.6535143277869793
0.7494972234853776
0.7494972234853776
0.7725570327602327
0.7725570327602327
0.7941068025393179
0.7941068025393179
0.8784923726198031
0.8784923726198031
0.898761630160152
0.898761630160152
0.9179738167295
0.9179738167295
0.994365549122449
0.994365549122449
1.0314029983564978
1.03140299835

1.1395465284294914
1.1616077713260893
1.1616077713260893
0.13410648569515615
0.13410648569515615
0.16623231357880286
0.16623231357880286
0.1961147155816434
0.1961147155816434
0.3094524465232098
0.3094524465232098
0.33639597888395656
0.33639597888395656
0.3608734994850238
0.3608734994850238
0.45685639763541985
0.45685639763541985
0.47991620680113556
0.47991620680113556
0.5014659772932646
0.5014659772932646
0.5858515500440262
0.5858515500440262
0.6061208085448015
0.6061208085448015
0.6253329953178763
0.6253329953178763
0.7017247295589186
0.7017247295589186
0.7387621803645743
0.7387621803645743
0.8063206780207111
0.8382766677823383
0.8382766677823383
0.8382766677823383
0.9588929826859385
0.9588929826859385
0.9874269255524268
0.9874269255524268
1.0134972421365092
1.0134972421365092
1.1152075056015747
1.1152075056015747
1.1395465284294914
1.1395465284294914
1.1616077713260893
1.1616077713260893
0.03212582686637688
0.03212582686637688
0.0620082279883718
0.0620082279883718
0.17534595594042912

0.47740319339209236
0.5980195130978245
0.5980195130978245
0.6265534588819719
0.6265534588819719
0.6526237764483085
0.6526237764483085
0.7543340435004211
0.7543340435004211
0.7786730689767865
0.7786730689767865
0.8007343136632699
0.8007343136632699
0.09598290363646811
0.09598290363646811
0.11904271378716658
0.11904271378716658
0.14059248614285025
0.14059248614285025
0.2249780624970299
0.2249780624970299
0.24524732185636822
0.24524732185636822
0.26445950940978946
0.26445950940978946
0.3408512477435579
0.3408512477435579
0.3778887007938465
0.3778887007938465
0.4454472022844129
0.47740319339209236
0.47740319339209236
0.47740319339209236
0.5980195130978245
0.5980195130978245
0.6265534588819719
0.6265534588819719
0.6526237764483085
0.6526237764483085
0.7543340435004211
0.7543340435004211
0.7786730689767865
0.7786730689767865
0.8007343136632699
0.8007343136632699
0.023059808719835928
0.023059808719835928
0.044609579726056836
0.044609579726056836
0.1289951506360012
0.1289951506360012
0.1492644

0.27693083213307546
0.3012698547227046
0.3012698547227046
0.3233310980085662
0.3233310980085662
0.12061631137021323
0.12061631137021323
0.14915025525715464
0.14915025525715464
0.17522057124097046
0.17522057124097046
0.27693083213307546
0.27693083213307546
0.3012698547227046
0.3012698547227046
0.3233310980085662
0.3233310980085662
0.12061631137021323
0.12061631137021323
0.14915025525715464
0.14915025525715464
0.17522057124097046
0.17522057124097046
0.27693083213307546
0.27693083213307546
0.3012698547227046
0.3012698547227046
0.3233310980085662
0.3233310980085662
0.02853394333237702
0.02853394333237702
0.054604259094503504
0.054604259094503504
0.15631451825902332
0.15631451825902332
0.18065354081772966
0.18065354081772966
0.20271478353788552
0.20271478353788552
0.02853394333237702
0.02853394333237702
0.054604259094503504
0.054604259094503504
0.15631451825902332
0.15631451825902332
0.18065354081772966
0.18065354081772966
0.20271478353788552
0.20271478353788552
0.026070316363075108
0.02607

NameError: name 'start_dkl1' is not defined